In [77]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")

# MCA_WT is Database name 
db = client["MCA_WT"]

In [78]:
# Collections
owners_collection = db["owners"]  # Collection for owners' information
properties_collection = db["properties"]  # Collection for property information

# Clear collections for a fresh start (optional)
owners_collection.delete_many({})  # Deletes all documents in the owners collection
properties_collection.delete_many({})  # Deletes all documents in the properties collection


DeleteResult({'n': 5, 'ok': 1.0}, acknowledged=True)

In [79]:
# Sample data insertion
# Insert owners
# Inserting sample documents into the owners collection, each document represents an owner with an ID, name, contact, and associated property IDs.
owners = [
    {"_id": 1, "name": "Mr. Patil", "contact": "1234567890", "properties": [1, 2]},
    {"_id": 2, "name": "Ms. Sharma", "contact": "0987654321", "properties": [3]},
    {"_id": 3, "name": "Mr. Singh", "contact": "1122334455", "properties": [4, 5]}
]
# Insert many -> it inserts many document/records 
owners_collection.insert_many(owners)  # Insert list of documents into the owners collection


InsertManyResult([1, 2, 3], acknowledged=True)

In [80]:
# Insert properties
# Inserting sample documents into the properties collection, each document represents a property with an ID, address, rate, area, and owner ID.
properties = [
    {"_id": 1, "address": {"area": "Nashik Road", "city": "Nashik"}, "rate": 75000, "area": 500, "owner_id": 1},
    {"_id": 2, "address": {"area": "Gangapur Road", "city": "Nashik"}, "rate": 1500, "area": 800, "owner_id": 1},
    {"_id": 3, "address": {"area": "Nashik Road", "city": "Nashik"}, "rate": 200000, "area": 1000, "owner_id": 2},
    {"_id": 4, "address": {"area": "Nashik Road", "city": "Nashik"}, "rate": 900, "area": 700, "owner_id": 1},
    {"_id": 5, "address": {"area": "Dadar", "city": "Mumbai"}, "rate": 85000, "area": 650, "owner_id": 3}
]
properties_collection.insert_many(properties)  # Insert list of documents into the properties collection


InsertManyResult([1, 2, 3, 4, 5], acknowledged=True)

In [81]:
# a. Display area-wise property details
print("a. Area-wise property details:")
# Using aggregate to group properties by area within the address field

# "$group": { "_id": "$address.area" }: Groups documents by the area field inside the address subdocument.
# "properties": { "$push": "$$ROOT" }: Pushes the entire document ($$ROOT) for each group into an array called properties.
# $push : Collects values and puts them into an array.
# $$ROOT : The entire document currently being processed.
# $$ Prefix: Indicates that it’s a special variable in MongoDB’s aggregation.
pipeline = [
    
    {
        "$group": {
            "_id": "$address.area",  # Group by the 'area' field inside the 'address' field
            "properties": {"$push": "$$ROOT"}  # Collect all documents for each area into a list
        }
    }
]
area_wise_pro_array = properties_collection.aggregate(pipeline)

# Iterate and print each group
for area in area_wise_pro_array:
    # print(area)
    print(" _id : ",area['_id'])
    print("\t Records : ")
    for r in area['properties']:
        print("\t\t ",r)
    


a. Area-wise property details:
 _id :  Gangapur Road
	 Records : 
		  {'_id': 2, 'address': {'area': 'Gangapur Road', 'city': 'Nashik'}, 'rate': 1500, 'area': 800, 'owner_id': 1}
 _id :  Nashik Road
	 Records : 
		  {'_id': 1, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 75000, 'area': 500, 'owner_id': 1}
		  {'_id': 3, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 200000, 'area': 1000, 'owner_id': 2}
		  {'_id': 4, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 900, 'area': 700, 'owner_id': 1}
 _id :  Dadar
	 Records : 
		  {'_id': 5, 'address': {'area': 'Dadar', 'city': 'Mumbai'}, 'rate': 85000, 'area': 650, 'owner_id': 3}


In [82]:
properties = [
    {"_id": 1, "address": {"area": "Nashik Road", "city": "Nashik"}, "rate": 75000, "area": 500, "owner_id": 1},
    {"_id": 2, "address": {"area": "Gangapur Road", "city": "Nashik"}, "rate": 150000, "area": 800, "owner_id": 1},
    {"_id": 3, "address": {"area": "Nashik Road", "city": "Mumbai"}, "rate": 200000, "area": 1000, "owner_id": 2},
    {"_id": 4, "address": {"area": "Nashik Road", "city": "Nashik"}, "rate": 90000, "area": 700, "owner_id": 3},
    {"_id": 5, "address": {"area": "Dadar", "city": "Mumbai"}, "rate": 85000, "area": 650, "owner_id": 3}
]

d = {}

for p in properties:
    if p['address']['area'] not in d:
        d[p['address']['area']] = [p]
    else:
        lst = d[p['address']['area']]
        lst.append(p)
        d[p['address']['area']] = lst

for r in d:
    print(r)
    for i in d[r]:
        print('\t ',i)

    



Nashik Road
	  {'_id': 1, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 75000, 'area': 500, 'owner_id': 1}
	  {'_id': 3, 'address': {'area': 'Nashik Road', 'city': 'Mumbai'}, 'rate': 200000, 'area': 1000, 'owner_id': 2}
	  {'_id': 4, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 90000, 'area': 700, 'owner_id': 3}
Gangapur Road
	  {'_id': 2, 'address': {'area': 'Gangapur Road', 'city': 'Nashik'}, 'rate': 150000, 'area': 800, 'owner_id': 1}
Dadar
	  {'_id': 5, 'address': {'area': 'Dadar', 'city': 'Mumbai'}, 'rate': 85000, 'area': 650, 'owner_id': 3}


In [83]:
# b. Display property owned by 'Mr. Patil' having minimum rate

print("\nb. Property owned by 'Mr. Patil' with minimum rate:")
# Finding Mr. Patil's document in the owners collection
mr_patil = owners_collection.find_one({"name": "Mr. Patil"})
if mr_patil:
    # Finding properties with Mr. Patil's ID and sorting by 'rate' in ascending order to get the minimum rate
    ownerId = mr_patil["_id"]
    min_rate_property = properties_collection.find({"owner_id": ownerId}).sort("rate", 1).limit(1)
    for property_ in min_rate_property:
        print(property_)
else:
    print("Mr. Patil not found in database.")


b. Property owned by 'Mr. Patil' with minimum rate:
{'_id': 4, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 900, 'area': 700, 'owner_id': 1}


In [91]:
# c. Details of owner whose property is at “Nashik”
print("\nc. Details of owner with property in Nashik:")
# Finding one property in Nashik city
property_in_nashik = properties_collection.find({"address.city": "Nashik"})


if property_in_nashik:
    print(" All properties of nashik")

    # for r in property_in_nashik:
    #     print(r)


    lst = []

    for p in property_in_nashik:
        print(p)
        lst.append(p['owner_id'])

    print('ALL owner id : ',lst)
    lst = set(lst)

    print('List of actual owner IDs : ',lst)
    for oid in lst:
        owner_in_nashik = owners_collection.find_one({"_id": oid})
        print(owner_in_nashik['name'])
else:
    print("No property found in Nashik.")


c. Details of owner with property in Nashik:
 All properties of nashik
{'_id': 1, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 75000, 'area': 500, 'owner_id': 1}
{'_id': 2, 'address': {'area': 'Gangapur Road', 'city': 'Nashik'}, 'rate': 1500, 'area': 800, 'owner_id': 1}
{'_id': 3, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 200000, 'area': 1000, 'owner_id': 2}
{'_id': 4, 'address': {'area': 'Nashik Road', 'city': 'Nashik'}, 'rate': 900, 'area': 700, 'owner_id': 1}
ALL owner id :  [1, 1, 2, 1]
List of actual owner IDs :  {1, 2}
Mr. Patil
Ms. Sharma


In [99]:
# d. Display area of property with rate less than 100000
print("\nd. Area of property with rate less than 100000:")
# Query to find properties where rate is less than 100000
properties_less_than_100000 = properties_collection.find(
    {"rate": {"$lt": 100000}},  # Filter condition for properties with rate < 100000
    {"address.area": 1,"address.city": 1,  "_id": 0}  # Project only the 'area' field inside 'address'; exclude '_id' from output
)

# find( { first it is for condition }, { whats to inclued in output })
# Iterate and print the areas of each property found
for property_ in properties_less_than_100000:
    print(property_)


d. Area of property with rate less than 100000:
{'address': {'area': 'Nashik Road', 'city': 'Nashik'}}
{'address': {'area': 'Gangapur Road', 'city': 'Nashik'}}
{'address': {'area': 'Nashik Road', 'city': 'Nashik'}}
{'address': {'area': 'Dadar', 'city': 'Mumbai'}}
